In [1]:
import numpy as np
from dsp.decoding import BiphaseDecoder, BlockGenerator

In [10]:
import warnings
warnings.filterwarnings('ignore')

import os
from os.path import join
root_path = os.getenv('RADIO_PATH', '.')

In [11]:
data_path = join(root_path, 'data/rds_physical_output.npy')
raw_bits = np.load(data_path)

In [27]:
bip_decoder = BiphaseDecoder()
bits = bip_decoder.run(raw_bits).astype(np.int)

In [17]:
gen = BlockGenerator()

In [28]:
prev = 0
for s in range(len(bits) - 26 + 1):
    syndrome = gen.syndrome(bits[s:s + 26])
    if syndrome in gen.offset_syndrome:
        print(s, s - prev, gen.offset_syndrome[syndrome])
        prev = s

123 123 2
254 131 2
671 417 2
694 23 3
707 13 2
987 280 1
1043 56 0
1144 101 1
1260 116 2
1322 62 2
1400 78 0
1436 36 3
2081 645 1
2335 254 1
2569 234 2
3274 705 2
3496 222 0
3510 14 2
4372 862 0
4490 118 3
4666 176 0
4767 101 2
4922 155 3
5647 725 3
5690 43 1
6140 450 0
6214 74 2
6647 433 3
7562 915 0
7786 224 3
7935 149 2
8165 230 0
8334 169 0
8405 71 2
8490 85 1
8670 180 2
8821 151 3
9190 369 2
9230 40 1
9233 3 3
9312 79 2
9562 250 2
9684 122 1
9837 153 3
10305 468 0
10313 8 2
10356 43 1
10382 26 0
10490 108 2
10553 63 1
10582 29 2
10772 190 2
11056 284 0
11198 142 3
11344 146 2
11384 40 1
11426 42 2
11446 20 2
11621 175 3
11656 35 0
11721 65 2
11830 109 2
12043 213 2
12107 64 2
12143 36 0
12214 71 2
12229 15 0
12265 36 2
12287 22 2
12762 475 2
12883 121 2
13036 153 2
13044 8 2
13399 355 3
13446 47 2
13994 548 2
14187 193 1
14592 405 0
14792 200 0
15668 876 0
15842 174 2
16217 375 0
16438 221 0
16543 105 0
16881 338 1
17058 177 2
17083 25 2
17222 139 0
17288 66 2
17850 562 0
17863 1

In [19]:
gen.synchronize(bits)

(2569, 2)

In [20]:
blocks = []
gen.process_blocks(bits[162:], blocks, 0)

1170

In [21]:
blocks = gen.find_blocks(bits)

('SYNCED: ', 2569)
('LOST SYNC: ', 3765)
('SYNCED: ', 6140)
('LOST SYNC: ', 7336)
('SYNCED: ', 9190)
('LOST SYNC: ', 10412)
('SYNCED: ', 11426)
('LOST SYNC: ', 12622)
('SYNCED: ', 16438)
('LOST SYNC: ', 17634)
('SYNCED: ', 18887)
('LOST SYNC: ', 20083)
('SYNCED: ', 21446)
('LOST SYNC: ', 22642)
('SYNCED: ', 23323)
('LOST SYNC: ', 24519)
('SYNCED: ', 25040)
('LOST SYNC: ', 25040)


In [22]:
[b for i, b in blocks if i == 0]

[array([0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1]),
 array([0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]),
 array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0]),
 array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]),
 array([0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])]

Results are pretty bad. Let's try it out on bits decoded by Redsea.

In [24]:
redsea_data_path = join(root_path, 'data/redsea_bits')

with open(redsea_data_path, 'r') as redsea:
    redsea_str = redsea.read().strip()
redsea_bitlist = list(map(int, redsea_str))
redsea_bits = np.array(redsea_bitlist, dtype=np.int)

In [25]:
blocks = gen.find_blocks(redsea_bits)

('SYNCED: ', 194)
('LOST SYNC: ', 23084)


Holy...

In [11]:
blocks

[(0, array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1])),
 (1, array([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (2, array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1])),
 (3, array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])),
 (1, array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1])),
 (2, array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])),
 (3, array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])),
 (0, array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1])),
 (1, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (2, array([1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0])),
 (3, array([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1])),
 (0, array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1])),
 (1, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1])),
 (2, array([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1])),
 (3, array([0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1])),
 (0, array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0

In [12]:
[(ind, b) for ind, (i, b) in enumerate(blocks) if i == 1]

[(1, array([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (4, array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1])),
 (8, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (12, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1])),
 (16, array([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (20, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0])),
 (24, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1])),
 (28, array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (32, array([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (36, array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1])),
 (40, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (44, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1])),
 (48, array([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0])),
 (52, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0])),
 (56, array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1])),
 (60, array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 